***
<p style="text-align:left;">Reinforcement Learning
<span style="float:right;">Monday, 18. May 2020</span></p>

<p style="text-align:left;">Prof. S. Harmeling
<span style="float:right;">DUE 23:55 Monday, 25. May 2020</span></p>

---
<p style="text-align:center;"><b>Exercise set #5</b></p>

---

# 3. Sarsa

In this exercise you will implement **Sarsa**, which is a temporal-difference control algorithm.

In [1]:
import gym
import numpy as np
from time import sleep
from IPython.display import clear_output
from gym.envs.toy_text.discrete import DiscreteEnv
from gym.wrappers.time_limit import TimeLimit

## Windy Gridworld Environment

We will apply Sarsa to the **Windy Gridworld with King's Moves** environment from [Reinforcement Learning: An Introduction](http://incompleteideas.net/book/the-book-2nd.html) by Sutton and Barto, p. 130,  
which we already implemented below. King's moves just means that we are also allowed to make diagonal moves.

In [2]:
class WindyGridworld(DiscreteEnv):
    
    UP = 0
    RIGHT = 1
    DOWN = 2
    LEFT = 3
    
    # king's moves
    UP_LEFT = 4
    UP_RIGHT = 5
    DOWN_LEFT = 6
    DOWN_RIGHT = 7
    
    metadata = {'render.modes': ['human']}
    
    def __init__(self, kings_moves=True):
        self.kings_moves = kings_moves
        self.shape = (7, 10)
        self.start_state = np.ravel_multi_index((3, 0), self.shape)
        self.goal_state = np.ravel_multi_index((3, 7), self.shape)
        self.wind = [0, 0, 0, 1, 1, 1, 2, 2, 1, 0]
        
        nS = np.prod(self.shape)
        nA = 8 if kings_moves else 4
        
        # transition probabilities
        P = {}
        for s in range(nS):
            position = np.unravel_index(s, self.shape)
            P[s] = {a: [] for a in range(nA)}
            P[s][self.UP] = self._transition_prob(position, (-1, 0))
            P[s][self.RIGHT] = self._transition_prob(position, (0, 1))
            P[s][self.DOWN] = self._transition_prob(position, (1, 0))
            P[s][self.LEFT] = self._transition_prob(position, (0, -1))
            if kings_moves:
                P[s][self.UP_LEFT] = self._transition_prob(position, (-1, -1))
                P[s][self.UP_RIGHT] = self._transition_prob(position, (-1, 1))
                P[s][self.DOWN_LEFT] = self._transition_prob(position, (1, -1))
                P[s][self.DOWN_RIGHT] = self._transition_prob(position, (1, 1))
        
        # initial state distribution
        isd = np.zeros(nS)
        isd[self.start_state] = 1.0
        
        self._last_state_action = {}
        
        super().__init__(nS, nA, P, isd)
    
    def _transition_prob(self, position, move):
        y, x = position
        wind = self.wind[x]
        new_position = (
            min(max(y + move[0] - wind, 0), self.shape[0] - 1),
            min(max(x + move[1], 0), self.shape[1] - 1)
        )
        
        prob = 1.0
        new_state = np.ravel_multi_index(new_position, self.shape)
        reward = -1
        done = new_state == self.goal_state
        return [(prob, new_state, reward, done)]
    
    def render(self, mode='human'):
        for state in range(self.nS):
            if state == self.s:
                output = ' x '
            elif state == self.start_state:
                output = ' S '
            elif state == self.goal_state:
                output = ' G '
            else:
                output = ' - '
            
            y, x = np.unravel_index(state, self.shape)
            if x == 0:
                output = output.lstrip()
            if x == self.shape[1] - 1:
                output = output.rstrip() + '\n'
            
            print(output, end='')
        print()

Now, let's create an instance of the environment.  
In gym we can *wrap* environments with wrapper classes, that can change the behavior of an environment.  
We will use this to limit the number of steps per episode to 1000, so the episodes don't get too long, e.g. if a policy does not reach the goal:

In [3]:
env = WindyGridworld(kings_moves=True)
env = TimeLimit(env, max_episode_steps=1000)

## Policies

In our setup, policies are functions that take two arguments, ```env``` and ```state```, and return an action based on that state:

```
def my_policy(env, state):
    action = ...
    return action
```

Below we implemented a function that runs one rollout of a given policy:

In [4]:
def rollout(env, policy, render=False):
    state = env.reset()
    total_reward = 0.
    done = False
    while not done:
        if render:
            env.render()
            clear_output(wait=True)
            
        action = policy(env, state)
        state, reward, done, info = env.step(action)
        total_reward += reward
        
        if render:
            sleep(0.4)
    
    if render:
        env.render()
    return total_reward

And another function that runs multiple rollouts of a given policy and averages the total rewards:

In [5]:
def evaluate(env, policy, num_rollouts=100):
    return sum(rollout(env, policy) for _ in range(num_rollouts)) / num_rollouts

### Random policy
A random policy selects random actions and ignores the current state.  
Let's see how this very simple policy performs on the environment:

In [6]:
def random_policy(env, state):
    return env.action_space.sample()

In [7]:
# limit the number of steps, to prevent a very long wait time
total_reward = rollout(TimeLimit(env, max_episode_steps=40), random_policy, render=True)
print('Total reward:', total_reward)

-  -  -  -  -  -  x  -  -  -
-  -  -  -  -  -  -  -  -  -
-  -  -  -  -  -  -  -  -  -
S  -  -  -  -  -  -  G  -  -
-  -  -  -  -  -  -  -  -  -
-  -  -  -  -  -  -  -  -  -
-  -  -  -  -  -  -  -  -  -

Total reward: -40.0


In [8]:
print('Average total reward:', evaluate(env, random_policy, num_rollouts=100))

Average total reward: -787.43


As we can see, the random policy performs poorly.

### Epsilon-greedy policies

In the Sarsa algorithm we will compute action-values $Q$, which need to be converted to actual policies.  

For example, a *greedy* policy chooses the actions with the highest action-values.  
This will be used for the action-values that Sarsa computed after it has *finished*.  
```greedy_policy_from_q``` creates a policy function, that greedily chooses actions:

In [9]:
def greedy_policy_from_q(Q):
    def policy(env, state):
        return np.argmax(Q[state])
    return policy

Inside Sarsa one typically uses *epsilon-greedy* policies, that choose random actions with probability $\epsilon$,  
and choose actions with the highest action-value with probability $1 - \epsilon$.  

This can lead to better results, because choosing random actions from time to time increases the *exploration* of the state space.

Implement the following function, that converts action-values to an epsilon-greedy policy:

In [10]:
def epsilon_greedy_policy_from_q(Q, epsilon):
    def policy(env, state):
        if np.random.rand() < epsilon:
            return np.random.choice(range(len(Q[state])))
        else:
            return np.argmax(Q[state])
    #########################
    # Write your code here. #
    #########################
    return policy

## Sarsa

Now we finally implement Sarsa, that tries to find the optimal action-value function $Q(s,a)$.

Implement the Sarsa algorithm from [Reinforcement Learning: An Introduction](http://incompleteideas.net/book/the-book-2nd.html) by Sutton and Barto, p. 130:

In [11]:
def sarsa(env, epsilon, alpha, gamma, num_episodes):
    Q = np.zeros((env.nS, env.nA))
    #########################
    # Write your code here. #
    for e in range(num_episodes):
        S = env.reset()
        done = False
        while True:
            policy = epsilon_greedy_policy_from_q(Q, epsilon)
            A = int(policy(env, S))
            assert A in range(env.nA), f'{A}, {env.nA}'
            S_, R, done, info = env.step(A)
            if done:
                break
            A_ = int(policy(env, S_))
            Q[S, A] += alpha*(R + gamma*(Q[S_, A_]) - Q[S, A])
            S = S_
    #########################
    return Q

We should run sarsa multiple times to have a greater chance of finding the optimal policy:

In [12]:
def run_sarsa(env, epsilon, alpha, gamma, num_episodes, num_runs):
    best_avg_total_reward = None
    best_Q = None
    best_policy = None
    
    # run num_runs times and store the policy which has the highest average total reward
    for _ in range(num_runs):
        Q = sarsa(env, epsilon, alpha, gamma, num_episodes)
        policy = greedy_policy_from_q(Q)
        
        avg_total_reward = evaluate(env, policy, num_rollouts=10)
        if best_avg_total_reward is None or avg_total_reward > best_avg_total_reward:
            best_avg_total_reward = avg_total_reward
            best_Q = Q
            best_policy = policy
    
    return best_Q, best_policy

In [13]:
Q, policy = run_sarsa(env, epsilon=0.1, alpha=0.5, gamma=1, num_episodes=200, num_runs=10)

In [14]:
total_reward = rollout(TimeLimit(env, max_episode_steps=40), policy, render=True)
print('Total reward:', total_reward)

-  -  -  -  -  -  -  -  -  -
-  -  -  -  -  -  -  -  -  -
-  -  -  -  -  -  -  -  -  -
S  -  -  -  -  -  -  x  -  -
-  -  -  -  -  -  -  -  -  -
-  -  -  -  -  -  -  -  -  -
-  -  -  -  -  -  -  -  -  -

Total reward: -7.0


In [15]:
print('Average total reward:', evaluate(env, policy, num_rollouts=100))

Average total reward: -7.0


### Visualizing the path
Let's visualize the path that the policy takes:

In [16]:
def visualize_path(env, policy):
    action_symbols = ['↑', '→', '↓', '←', '↖', '↗', '↙', '↘']
    
    state_actions = {}
    done = False
    state = env.reset()
    while not done:
        action = policy(env, state)
        state_actions[state] = action
        state, reward, done, info = env.step(action)
    terminal_state = state
    
    for state in range(env.nS):
        if state == terminal_state:
            output = ' x '
        elif state in state_actions:
            action = state_actions[state]
            output = ' ' + action_symbols[action] + ' '
        else:
            output = ' - '
        
        y, x = np.unravel_index(state, env.shape)
        if x == 0:
            output = output.lstrip()
        if x == env.shape[1] - 1:
            output = output.rstrip() + '\n'
        
        print(output, end='')
    print()

In [17]:
visualize_path(env, policy)

-  -  -  -  -  -  -  -  -  -
-  -  -  -  -  -  -  -  -  -
-  -  -  -  -  -  -  -  -  -
↘  -  -  -  -  -  -  x  -  -
-  ↘  -  -  -  -  -  -  -  -
-  -  ↘  -  -  -  →  -  -  -
-  -  -  ↘  ↘  →  -  -  -  -



### Without King's moves
Let's see what happens, if we disable diagonal moves:

In [18]:
env_straight = TimeLimit(WindyGridworld(kings_moves=False), max_episode_steps=1000)
Q_straight, policy_straight = run_sarsa(env_straight, epsilon=0.1, alpha=0.5, gamma=1, num_episodes=200, num_runs=10)

In [19]:
total_reward = rollout(env_straight, policy_straight, render=True)
print('Total reward:', total_reward)

-  -  -  -  -  -  -  -  -  -
-  -  -  -  -  -  -  -  -  -
-  -  -  -  -  -  -  -  -  -
S  -  -  -  -  -  -  x  -  -
-  -  -  -  -  -  -  -  -  -
-  -  -  -  -  -  -  -  -  -
-  -  -  -  -  -  -  -  -  -

Total reward: -15.0


In [20]:
print('Average total reward:', evaluate(env_straight, policy_straight, num_rollouts=100))

Average total reward: -15.0


In [21]:
visualize_path(env_straight, policy_straight)

-  -  -  -  -  -  →  →  →  ↓
-  -  -  -  -  →  -  -  -  ↓
-  -  -  -  →  -  -  -  -  ↓
→  →  →  →  -  -  -  x  -  ↓
-  -  -  -  -  -  -  -  ←  ←
-  -  -  -  -  -  -  -  -  -
-  -  -  -  -  -  -  -  -  -

